In [2]:
# importing required modules

import os
import util
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from numpy import linalg as LA
import random
import os.path,subprocess
import time

In [3]:
# loading the model
df = util.get_processed_data("./../data/collections_math.csv", False)
collections = util.get_collections(df)

In [4]:
#  function to get SC and NV scores
def getNoveltySC(text1,text2):
    cmd =['java','-jar','NoveltySemanticCoherence.jar','SinglePair',text1,text2]
    subprocess.call(cmd)
    f = open("./data/pairwise/singlepairwise", "r")
    sc =float(f.readline()) 
    nv = float(f.readline())
    return sc,nv

In [5]:
# df["collection_id"].unique()

In [5]:
# novelty testing function which takes an collection id as input and test on that collection 

def novelty_testing(col_id):
    test = df[df["collection_id"]==col_id]
    test = test.sort_values(by=["sequence_id"])
    temp = test.text
    texts = [] 
    for i in temp :
        texts.append(i)
    adjacent = []
    
    for i in range(0,len(texts)-1):
        adjacent.append(getNoveltySC(texts[i],texts[i+1]))

    non_adjacent_one = []    
    for i in range (0,len(texts)-2):
        non_adjacent_one.append(getNoveltySC(texts[i],texts[i+2]))

    non_adjacent_two = []    
    for i in range(0,len(texts) -3):
        non_adjacent_two.append(getNoveltySC(texts[i],texts[i+3]))

    return adjacent,non_adjacent_one,non_adjacent_two

In [1]:
#  testing on a valid collection and writing the scores into a file

adj,nadj1,nadj2 = novelty_testing("001bf2c6-8ede-478a-9b8b-d7750488cb1b")
test = df[df["collection_id"]=="001bf2c6-8ede-478a-9b8b-d7750488cb1b"]
test = test.sort_values(by=["sequence_id"])
temp = test.text

f = open("./../001bf2c6-8ede-478a-9b8b-d7750488cb1b.txt", "w")

for i in temp :
        f.write("\n---------------------------------\n")
        f.write(i)
f.write("\n---------------------------------\n")
f.write(str(adj))
f.write("\n---------------------------------\n")
f.write(str(nadj1))
f.write("\n---------------------------------\n")
f.write(str(nadj2))
f.close()


In [6]:
#  generating an incvalid collection and test on it and writing the scores into a file

random_sample = df.text.sample(n = 10)
texts = []


f = open("./../random_collection.txt", "w")

for i in random_sample:
    f.write("\n---------------------------------------------\n")
    f.write(i)
    texts.append(i)

adjacent = []
for i in range(0,len(texts)-1):
    adjacent.append(getNoveltySC(texts[i],texts[i+1])[1])

non_adjacent = []    
for i in range (0,len(texts)-2):
    non_adjacent.append(getNoveltySC(texts[i],texts[i+2])[1])

non_adjacent1 = []    
for i in range (0,len(texts)-3):
    non_adjacent1.append(getNoveltySC(texts[i],texts[i+3])[1])

f.write("\n--------------------------------------------------\n")
f.write(str(adjacent))
f.write("\n--------------------------------------------------\n")
f.write(str(non_adjacent))
f.write("\n--------------------------------------------------\n")
f.write(str(non_adjacent1))
f.close()